In [7]:
# Imports and Loads

In [2]:
from base import *

In [3]:
ATLANTICO_NORTE = AtlanticoNorte()
ATLANTICO_SUL = AtlanticoSul()
PACIFICO_SUL = PacificoSul()
NINO = Nino()
COORDENADAS = Coordenadas()
PRECIPITACAO = Precipitacao()

In [10]:
# Check Data Structures

In [10]:
ATLANTICO_NORTE.data.shape[1], ATLANTICO_SUL.data.shape, PACIFICO_SUL.data.shape, NINO.data.shape, PRECIPITACAO.data.shape

(20, (468, 40), (468, 35), (468, 5), (468, 25))

In [11]:
def check_dimensions(datas):
    for r in datas:
        print("{}   -  Rows: {}   Columns: {}".format(r.name, r.shape[0], r.shape[1]))
        print()

In [12]:
def check_nan_fields(datas):
    for r in datas:
        print(str(r.name)+"  -  "+str(sum(r.isna().sum().values)))
        print()

In [8]:
check_nan_fields([ATLANTICO_NORTE.data, ATLANTICO_SUL.data, PACIFICO_SUL.data, NINO.data, COORDENADAS.data, PRECIPITACAO.data])

ATLANTICO_NORTE  -  0

ATLANTICO_SUL  -  0

PACIFICO_SUL  -  0

NINO  -  0

COORDENADAS  -  0

PRECIPITACAO  -  0



In [13]:
check_dimensions([ATLANTICO_NORTE.data, ATLANTICO_SUL.data, PACIFICO_SUL.data, NINO.data, COORDENADAS.data, PRECIPITACAO.data])

ATLANTICO_NORTE   -  Rows: 468   Columns: 20

ATLANTICO_SUL   -  Rows: 468   Columns: 40

PACIFICO_SUL   -  Rows: 468   Columns: 35

NINO   -  Rows: 468   Columns: 5

COORDENADAS   -  Rows: 24   Columns: 3

PRECIPITACAO   -  Rows: 468   Columns: 25

